In [48]:
import numpy as np
import pandas as pd
import scipy.stats as ss
from sklearn.preprocessing import StandardScaler

In [49]:
train  = pd.read_csv('competition_data/train.csv')
test  = pd.read_csv('competition_data/test.csv')
## Input Feature - Target 분리
##train_x = train.drop(['nerdiness', 'index'],axis=1)   ## Input Feature 값
##train_y = train['nerdiness']     

In [50]:
## Test 문항에 대한 Column
test_cols = ['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8','Q9','Q10',
             'Q11','Q12','Q13','Q14','Q15','Q16','Q17','Q18','Q19','Q20',
             'Q21','Q22', 'Q23','Q24','Q25','Q26']

## 응시 시간 문항에 대한 Column
time_cols = ['introelapse', 'testelapse', 'surveyelapse']

## TIPI 문항에 대한 Column
tipi_cols = ['TIPI1','TIPI2','TIPI3','TIPI4','TIPI5','TIPI6','TIPI7','TIPI8',
             'TIPI9','TIPI10']

vcl_exist = ['VCL1','VCL2','VCL3','VCL4','VCL5','VCL7','VCL8','VCL10','VCL11','VCL13','VCL14','VCL15','VCL16']
vcl_no_exist = ['VCL6','VCL9','VCL12']
## VCL 문항에 대한 Column
vcl_cols = vcl_exist + vcl_no_exist

## 설문문항에 대한 Column
survy_cols = ['country','education','urban','gender','engnat','age','hand','religion',
              'orientation','voted','married','familysize','ASD']

In [51]:
def eraseOutlis(dframe):
    tmp_frame = dframe.copy()
    tmp_no_outli = tmp_frame[tmp_frame['age'] < 100]
    
    intr_92q = tmp_no_outli['introelapse'].quantile(q=0.92,interpolation='nearest')
    tmp_no_outli=tmp_no_outli[tmp_no_outli['introelapse'] < intr_92q]
    
    ## testelapse 하위0.04% 미만의 값을 갖는 Row 들 전부 삭제
    test_004q = tmp_no_outli['testelapse'].quantile(q=0.0004,interpolation='nearest')
    tmp_no_outli=tmp_no_outli[tmp_no_outli['testelapse'] >= test_004q]

    ## surveyelapse 하위 0.2% 미만의 값을 갖는 Row 들 전부 삭제
    srvy_04q = tmp_no_outli['surveyelapse'].quantile(q=0.002,interpolation='nearest')
    tmp_no_outli=tmp_no_outli[tmp_no_outli['surveyelapse'] >= srvy_04q]

    ## familysize 에 극단적으로 큰값(2919) 를 갖는 Row 제거
    tmp_no_outli=tmp_no_outli[tmp_no_outli['familysize'] < 39]
    
    return tmp_no_outli
    

In [52]:
train_no_outli = eraseOutlis(train)

In [53]:
def replaceNAN(dframe):
    continuous_nan = ['age','introelapse','testelapse','surveyelapse']             ## 연속형 변수
    norminal_nan = list(dframe.columns.values)                         
    for cont in continuous_nan:
        norminal_nan.remove(cont)
        
    tmp_frame = dframe.copy()
    ## 연속형 변수들에 대해서는 평균값으로 대체
    tmp_frame[continuous_nan] = tmp_frame[continuous_nan].fillna(round(tmp_frame[continuous_nan].mean()))
    ## 이산형 변수들에 대해서는 최빈값으로 대체
    tmp_frame[norminal_nan] =tmp_frame[norminal_nan].fillna(tmp_frame[norminal_nan].mode().iloc[0].squeeze())
    return tmp_frame

In [54]:
train_no_outli_nan = replaceNAN(train_no_outli)

In [55]:
def calcTIPI(dframe):
    tp_enc = dframe.copy()
    tp_enc['tp_extra'] = (dframe['TIPI1'] + (8-dframe['TIPI6']))/2 ## 외향성 
    tp_enc['tp_agree'] = (dframe['TIPI7'] + (8-dframe['TIPI2']))/2 ## 친화성
    tp_enc['tp_consc'] = (dframe['TIPI3'] + (8-dframe['TIPI8']))/2 ## 성실성
    tp_enc['tp_emoti'] = (dframe['TIPI9'] + (8-dframe['TIPI4']))/2 ## 정서적 안정성
    tp_enc['tp_opens'] = (dframe['TIPI5'] + (8-dframe['TIPI10']))/2 ## 경험에 대한 개방성
    
    return tp_enc

In [56]:
def addColumn(dframe):
    tmp_frame=dframe.copy()
    tmp_frame['test_score'] = dframe[test_cols].sum(axis=1)
    tmp_frame['exist_know'] = dframe[vcl_exist].sum(axis=1)
    tmp_frame['mean_test'] = dframe[['testelapse']].sum(axis=1)/26   ## 테스트 문항 별 평균적 응답시간 추가
    tmp_frame['mean_srvy'] = dframe[['surveyelapse']].sum(axis=1)/39 ## 설문 문항 별 평균적 응답시간 추가
    #tmp_frame['age_log'] = np.log(dframe['age'])                   ## age 에 로그를 취하여, 정규화
    tmp_frame = calcTIPI(tmp_frame)
    
    return tmp_frame

def dropColumn(dframe):
    drop_Qs = test_cols.copy()
    drop_Qs.remove('Q15')
    drop_Qs.remove('Q25')
    droplist =  drop_Qs + ['familysize','age','orientation','hand','married','ASD'] + vcl_cols + tipi_cols + time_cols
    tmp_frame = dframe.copy()
    tmp_frame = tmp_frame.drop(droplist, axis=1)
    return tmp_frame

In [57]:
## 이상치 제거, 결측치 처리된 Dataframe 대상으로 새로운 Column 들 추가 후, 기존 일부 Column들 제거
train_final_add = addColumn(train_no_outli_nan)
train_final_drop = dropColumn(train_final_add)

In [58]:
def ratio_table(cross_tab):
    return cross_tab.div(cross_tab.sum(axis=1),axis='index')

def groupCntryCol(dframe):
    tmp_country = train[['country','nerdiness']].dropna()
    native_country_table=tmp_country['country'].value_counts()
    country_count = tmp_country['country'].unique().shape[0]
    
    country_crosstab=pd.crosstab(tmp_country['country'],tmp_country['nerdiness'])
    country_crossgtab_res=ss.chi2_contingency(country_crosstab)
    
    country_ratio=ratio_table(country_crosstab)
    country_ratio=country_ratio.sort_values(by=0)
    
    prob_dict=dict(country_ratio[1])
    
    cntry_group1=[]
    cntry_group2=[]
    cntry_group3=[]
    cntry_group4=[]

    for country in tmp_country['country'].unique():
        country_data=tmp_country[tmp_country['country']==country]['nerdiness']
        probs=sum(country_data)/country_data.count()
        if probs <= 1 and probs >=0.6:
            cntry_group1.append(country)
        elif probs >=0.4 and probs <0.6:
            cntry_group2.append(country)
        elif probs >= 0.2 and probs < 0.4:
            cntry_group3.append(country)
        else:
            cntry_group4.append(country)
    #print(country,probs)
    return (cntry_group1, cntry_group2, cntry_group3, cntry_group4)


def groupCountry(dframe,group):
    tmp_frame = dframe.copy()
    
    tmp_frame['country']=tmp_frame['country'].replace(group[0],0)
    tmp_frame['country']=tmp_frame['country'].replace(group[1],1)
    tmp_frame['country']=tmp_frame['country'].replace(group[2],2)
    tmp_frame['country']=tmp_frame['country'].replace(group[3],3)
    
    return tmp_frame

In [59]:
cntry_group1, cntry_group2, cntry_group3, cntry_group4 = groupCntryCol(train_final_drop)
train_final_grouped = groupCountry(train_final_drop, (cntry_group1, cntry_group2, cntry_group3, cntry_group4))

In [60]:
def one_hot_encoding(dframe):
    tmp_frame = dframe.copy()
    categorical_cols = ['country','urban','gender','engnat','religion','voted']
    
    for col in categorical_cols:
        col_ohe = pd.get_dummies(tmp_frame[col], prefix=col)
        tmp_frame = pd.concat((tmp_frame, col_ohe), axis=1).drop(col, axis=1)
    return tmp_frame

In [61]:
train_final_ohe = one_hot_encoding(train_final_grouped)

In [62]:
def stdize_cols(dframe):
    tmp_frame =dframe.copy()
    col_stdize = ['mean_test', 'mean_srvy']
    scaler = StandardScaler()
    numeric_df = pd.DataFrame(scaler.fit_transform(tmp_frame[col_stdize]),columns=col_stdize,index=tmp_frame.index)
    X_train_new = tmp_frame.drop(columns=col_stdize)
    train_final_ohe_stdize =pd.concat([numeric_df,X_train_new],axis=1)
    return train_final_ohe_stdize

In [63]:
train_final_ohe_stdize = stdize_cols(train_final_ohe)

In [64]:
def prepare_test(dframe):
    test_frame =dframe.copy()
    test_no_outli = eraseOutlis(test_frame)
    test_no_outli_nan = replaceNAN(test_no_outli)
    
    test_final = addColumn(test_no_outli_nan)
    test_final = dropColumn(test_final)
    test_final= groupCountry(test_final,(cntry_group1, cntry_group2, cntry_group3, cntry_group4))
    
    test_final_ohe = one_hot_encoding(test_final)
    test_final_ohe_stdize = stdize_cols(test_final_ohe)
    
    return test_final_ohe_stdize

In [65]:
test_pre = prepare_test(test)                   ## 타겟 변수값

In [66]:
train_x = train.drop(['nerdiness', 'index'],axis=1)   ## Input Feature 값
train_y = train['nerdiness']   

test_x = test_pre.drop(['index'],axis=1)   ## Input Feature 값